In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
dataset=pd.read_csv("CKD.csv")
dataset=pd.get_dummies(dataset,drop_first=True)

In [5]:
dataset

,age,bp,al,su,bgr,bu,sc,sod,pot,hrmo,...,pc_normal,pcc_present,ba_present,htn_yes,dm_yes,cad_yes,appet_yes,pe_yes,ane_yes,classification_yes
0,2.000000,76.459948,3.0,0.0,148.112676,57.482105,3.077356,137.528754,4.627244,12.518156,...,False,False,False,False,False,False,True,True,False,True
1,3.000000,76.459948,2.0,0.0,148.112676,22.000000,0.700000,137.528754,4.627244,10.700000,...,True,False,False,False,False,False,True,False,False,True
2,4.000000,76.459948,1.0,0.0,99.000000,23.000000,0.600000,138.000000,4.400000,12.000000,...,True,False,False,False,False,False,True,False,False,True
3,5.000000,76.459948,1.0,0.0,148.112676,16.000000,0.700000,138.000000,3.200000,8.100000,...,True,False,False,False,False,False,True,False,True,True
4,5.000000,50.000000,0.0,0.0,148.112676,25.000000,0.600000,137.528754,4.627244,11.800000,...,True,False,False,False,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,51.492308,70.000000,0.0,0.0,219.000000,36.000000,1.300000,139.000000,3.700000,12.500000,...,True,False,False,False,False,False,True,False,False,True
395,51.492308,70.000000,0.0,2.0,220.000000,68.000000,2.800000,137.528754,4.627244,8.700000,...,True,False,False,True,True,False,True,False,True,True
396,51.492308,70.000000,3.0,0.0,110.000000,115.000000,6.000000,134.000000,2.700000,9.100000,...,True,False,False,True,True,False,False,False,False,True
397,51.492308,90.000000,0.0,0.0,207.000000,80.000000,6.800000,142.000000,5.500000,8.500000,...,True,False,False,True,True,False,True,False,True,True


In [7]:
independent=dataset[['age', 'bp', 'al', 'su', 'bgr', 'bu', 'sc', 'sod', 'pot', 'hrmo', 'pcv','wc', 'rc', 'sg_b', 'sg_c', 'sg_d', 'sg_e', 
                     'rbc_normal', 'pc_normal','pcc_present', 'ba_present', 'htn_yes', 'dm_yes', 'cad_yes','appet_yes', 'pe_yes', 'ane_yes']]
dependent=dataset[['classification_yes']]

In [9]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(independent,dependent,test_size=0.30,random_state=0)

In [11]:
#standardisation
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [13]:
dataset['classification_yes'].value_counts()

classification_yes
True     249
False    150
Name: count, dtype: int64

In [15]:
from sklearn.linear_model import LogisticRegression

In [19]:
from sklearn.model_selection import GridSearchCV
param_grid={'solver':['newton-cg','lbfgs','liblinear','saga'],'penalty':['l2']}
grid= GridSearchCV(LogisticRegression(),param_grid,refit=True,verbose=3,n_jobs=-1,scoring='f1_weighted')
grid.fit(x_train,y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


C:\Users\keert\Anaconda3\Lib\site-packages\sklearn\utils\validation.py:1339: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'penalty': ['l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga']},
             scoring='f1_weighted', verbose=3)

In [21]:
re=grid.cv_results_
grid_predictions=grid.predict(x_test)
from sklearn.metrics import confusion_matrix
cm=confusion_matrix(y_test,grid_predictions)
from sklearn.metrics import classification_report
clf_report=classification_report(y_test,grid_predictions)

In [23]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print ("The f1-macro value for best parameter {}:". format(grid.best_params_),f1_macro)
print ("The confusion matrix:\n",cm)
print ("The report:\n",clf_report)

The f1-macro value for best parameter {'penalty': 'l2', 'solver': 'newton-cg'}: 0.9916844900066377
The confusion matrix:
 [[45  0]
 [ 1 74]]
The report:
               precision    recall  f1-score   support

       False       0.98      1.00      0.99        45
        True       1.00      0.99      0.99        75

    accuracy                           0.99       120
   macro avg       0.99      0.99      0.99       120
weighted avg       0.99      0.99      0.99       120



In [25]:
#roc-auc
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,1.450719,0.009997,0.029382,0.001676,l2,newton-cg,"{'penalty': 'l2', 'solver': 'newton-cg'}",0.982221,1.000000,0.982221,1.000000,1.000000,0.992888,0.008710,1
1,1.448129,0.004857,0.028586,0.001038,l2,lbfgs,"{'penalty': 'l2', 'solver': 'lbfgs'}",0.982221,1.000000,0.982221,1.000000,1.000000,0.992888,0.008710,1
2,0.015693,0.008037,0.022829,0.007288,l2,liblinear,"{'penalty': 'l2', 'solver': 'liblinear'}",0.964572,0.982221,0.964572,0.964572,0.981894,0.971566,0.008567,4
3,0.029983,0.004369,0.011889,0.002319,l2,saga,"{'penalty': 'l2', 'solver': 'saga'}",0.964572,0.982221,0.964572,0.982221,0.981894,0.975096,0.008594,3


In [27]:
grid.predict([[15,60,3,0,86,15,0.6,138,4,11,33,7700,3.8,0,0,0,0,1,1,0,0,1,1,0,1,0,0]])

array([False])

In [29]:
h=roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])
print(h)

1.0


In [31]:
grid.predict([[15,80,0,0,93,17,0.9,136,3.9,16.7,50,6200,5.2,1,0,0,0,1,1,0,0,0,0,0,1,0,0]])

array([False])